## 1. サンプル画像データのダウンロード
sequential DNA/RNA/IF-FISHによって取得されたサンプル画像データは以下にアップロードされている。
https://zenodo.org/records/10565509

以下のコードを実行し、データをダウンロードする。

In [4]:
!pip install zenodo_get
!zenodo_get --doi=10.5281/zenodo.10565509

Title: Sample image data obtained by sequential DNA/RNA/IF-FISH method
Keywords: 
Publication date: 2024-01-24
DOI: 10.5281/zenodo.10565024
Total size: 11964.6 MB

Link: https://zenodo.org/record/10565024/files/0_raw_data.zip   size: 11964.6 MB
100% [..............................................] 12545838909 / 1254583890976 / 1254583890940 / 1254583890940 / 1254583890988 / 1254583890992 / 1254583890984 / 1254583890904 / 1254583890964 / 1254583890940 / 1254583890904 / 1254583890972 / 1254583890908 / 1254583890992 / 1254583890936 / 1254583890976 / 12545838909     ] 10567401472 / 1254583890900 / 1254583890976 / 1254583890988 / 1254583890992 / 1254583890992 / 1254583890964 / 1254583890952 / 1254583890956 / 1254583890908 / 1254583890952 / 1254583890916 / 12545838909     ] 10669350912 / 1254583890984 / 1254583890992 / 12545838909     ] 10707673088 / 1254583890924 / 1254583890972 / 1254583890972 / 1254583890988 / 1254583890920 / 1254583890916 / 1254583890904 / 1254583890992 / 1254583890956 /

上記でダウンロードしたzipファイルを上の階層へ移動させ、unzipする。フォルダ名は"0_raw_data"のままにする。

次に、以下の" "の間に、githubからダウンロードしたフォルダのパスを指定する。

In [5]:
import os
import shutil
import zipfile

# 移動させるzipファイルの名前
zip_file = '0_raw_data.zip'

# 現在のディレクトリのパスを取得
current_directory = os.getcwd()

# zipファイルの完全なパス
full_zip_path = os.path.join(current_directory, zip_file)

# 一つ上の階層のディレクトリのパス
parent_directory = os.path.dirname(current_directory)

# zipファイルを一つ上の階層に移動
shutil.move(full_zip_path, parent_directory)

# 移動したzipファイルの新しいパス
new_zip_path = os.path.join(parent_directory, zip_file)

# zipファイルを解凍
with zipfile.ZipFile(new_zip_path, 'r') as zip_ref:
    zip_ref.extractall(parent_directory)

# 元のzipファイルおよび解凍に伴い出現する不要なファイルを削除
os.remove(new_zip_path)
shutil.rmtree(os.path.join(parent_directory, "__MACOSX"))

print(f"ファイル '{zip_file}' が一つ上の階層に移動され、解凍され、削除されました。")



# dir = "/Volumes/Public-TS-873/Microscope/Ochiai/240301-seqFISH-MMB/231224-image_analysis_v3"
# 現在のディレクトリの2つ上の階層のパスを取得
dir = os.path.dirname(current_directory)
print(dir)

ファイル '0_raw_data.zip' が一つ上の階層に移動され、解凍されました。
/Users/HiroshiOchiai/Library/CloudStorage/OneDrive-広島大学/ドキュメント/Nanog-Long-range-interaction/論文/2024-seqFISH-MMB


## PyImageJを利用した画像処理
ImageJを利用した画像処理を実施する。画像サイズが大きい場合、メモリが足りず処理がクラッシュする場合がある。そのため、十分なメモリを持つPCで実施する必要がある。
また、Javaを利用するが、事前にどの程度のメモリを使用するかを指定する必要がある。

Replace 6g with the amount of memory Java should have. Save some memory for your core operating system and other programs, though; a good rule of thumb is to give Java no more than 80% of your physical RAM.

plugins_dirとimagej.initについて、指定する必要がある。詳細は~~を参照のこと。

In [2]:
import imagej
import scyjava
import os
scyjava.config.add_option('-Xmx100g')
plugins_dir = '/Applications/Fiji_newer_220618/Fiji.app/plugins'
scyjava.config.add_option(f'-Dplugins.dir={plugins_dir}')
ij = imagej.init('/Applications/Fiji_newer_220618/Fiji.app')


 --- Setting global cache 
Global cache set to 22755 Mb, out of 45511 Mb available (50%)


以下を実行し、ImageJのバージョンが表示されれば問題なく実行できている。

In [3]:
print(ij.getVersion())

2.14.0/1.54f


以下のマクロでは以下の操作を実行します。
* バッチモードの設定: 処理の高速化のためにバッチモードが有効にされています。
* イメージファイルの読み込み: 特定のディレクトリから複数のイメージファイル(seqFISH_*.tif)を読み込みます。
* イメージの結合: 読み込まれたイメージを結合して、スタックにします。
* ハイパースタックへの変換: スタックをハイパースタックに変換し、チャネル、スライス、フレーム、ピクセルのプロパティを設定します。
* チャネルの分割と32ビット化: チャネルを分割し、それぞれを32ビットのイメージに変換します。
* フラットフィールドイメージとダークイメージの読み込み: 補正用のイメージファイルを読み込みます。
* ダークイメージによる補正: 各チャネルからダークイメージを差し引きます。
* フラットフィールドイメージによる正規化: 0になる数値を避けるために1を加算し、その後フラットフィールドイメージで割ります。
* チャネルのマージ: 4つのチャネルをマージして、新しいハイパースタックを作成します。
* 3Dドリフトの補正: 3Dドリフトを補正します。
* 保存: 処理済みのイメージを指定されたディレクトリに保存します。

In [ ]:
import os

macro = """
#@ String dir
#@ int cycle
#@ int Pos
#@ int zslices
#@ float pixel_width
#@ float voxel_depth


setBatchMode(true);
//バッチモード

for (i = 1; i < cycle+1; i++) {
	open(dir+"/0_raw_data/raw_data/seqFISH_"+Pos+"_"+IJ.pad(i, 3)+".tif");
	}
	
run("Concatenate...", "all_open title=Stack open");
run("Stack to Hyperstack...", "order=xyczt(default) channels=4 slices="+zslices+" frames="+cycle+" display=Composite");
run("Properties...", "channels=4 slices="+zslices+" frames="+cycle+" pixel_width="+pixel_width+" pixel_height="+pixel_width+" voxel_depth="+voxel_depth+"");
run("Conversions...", " ");
run("Split Channels");

selectImage("C1-Stack");
run("32-bit");
selectImage("C2-Stack");
run("32-bit");
selectImage("C3-Stack");
run("32-bit");
selectImage("C4-Stack");
run("32-bit");

//flat field imageを開く
open(dir+"/0_raw_data/flat_field_image/AB9-647.tif");
open(dir+"/0_raw_data/flat_field_image/Cou-405.tif");
open(dir+"/0_raw_data/flat_field_image/FL-488.tif");
open(dir+"/0_raw_data/flat_field_image/RB-561.tif");
// dark imageを開く
open(dir+"/0_raw_data/flat_field_image/DarkImage.tif");


// 各チャネルからdark imageを引く。
imageCalculator("Subtract 32-bit stack", "C1-Stack","DarkImage.tif");
imageCalculator("Subtract 32-bit stack", "C2-Stack","DarkImage.tif");
imageCalculator("Subtract 32-bit stack", "C3-Stack","DarkImage.tif");
imageCalculator("Subtract 32-bit stack", "C4-Stack","DarkImage.tif");
selectWindow("DarkImage.tif");
close();

// 数値が0になる場所が出てくるんので、全体に1を足しておく。さらにnormalized flat field imageで割り込む。
//ダメ押しでさらに1を足しておく。
selectWindow("C1-Stack");
run("Add...", "value=1 stack");
imageCalculator("Divide 32-bit stack", "C1-Stack","AB9-647.tif");
run("16-bit");
run("Add...", "value=1 stack");
selectWindow("AB9-647.tif");
close();

selectWindow("C2-Stack");
run("Add...", "value=1 stack");
imageCalculator("Divide 32-bit stack", "C2-Stack","RB-561.tif");
run("16-bit");
run("Add...", "value=1 stack");
selectWindow("RB-561.tif");
close();

selectWindow("C3-Stack");
run("Add...", "value=1 stack");
imageCalculator("Divide 32-bit stack", "C3-Stack","FL-488.tif");
run("16-bit");
run("Add...", "value=1 stack");
selectWindow("FL-488.tif");
close();

selectWindow("C4-Stack");
run("Add...", "value=1 stack");
imageCalculator("Divide 32-bit stack", "C4-Stack","Cou-405.tif");
run("16-bit");
run("Add...", "value=1 stack");
// Hoechst channelのみbreach correctionして、輝度値を揃えておく。
run("Bleach Correction", "correction=[Simple Ratio] background=0");
selectWindow("C4-Stack");
close();
selectWindow("DUP_C4-Stack");
rename("C4-Stack");
selectWindow("Cou-405.tif");
close();



//ここで4色をマージするが、Nuc channelを1にする。
run("Merge Channels...", "c1=C4-Stack c2=C1-Stack c3=C2-Stack c4=C3-Stack create ignore");

run("Stack to Hyperstack...", "order=xyczt(default) channels=4 slices="+zslices+" frames="+cycle+" display=Composite");
run("Properties...", "channels=4 slices="+zslices+" frames="+cycle+" pixel_width="+pixel_width+" pixel_height="+pixel_width+" voxel_depth="+voxel_depth+"");
rename("Composite");

// correct 3D drift
run("Correct 3D drift", "channel=1 edge_enhance only=0 lowest=1 highest=51 max_shift_x=50 max_shift_y=50 max_shift_z=30");
selectWindow("Composite");
close();
selectWindow("registered time points");
rename("Stack");


// 各視野を別々に保存する。


for (i = 1; i < cycle+1; i++) {
	selectWindow("Stack");
	run("Duplicate...", "title=Stack duplicate frames="+i+"");
	rename("to_be_saved");
	saveAs("Tiff", dir+"/1_processed_images/1_FL_corrected/Pos"+IJ.pad(Pos, 2)+"/ND_"+IJ.pad(i, 2)+"");
	close();
}

selectWindow("Stack");
close("*");

run("Quit");
"""

# Pos の値が 1 から 3 になるように繰り返す
for Pos in range(1, 4):  # range(1, 5) は 1, 2, 3, 4 の値を生成します
	# ImageJ マクロの 'IJ.pad(Pos, 2)' に相当する部分
	Pos_padded = str(Pos).zfill(2)

	# ディレクトリのパスを組み立てる
	directory_path = os.path.join(dir, "1_processed_images", "1_FL_corrected", "Pos" + Pos_padded)
	# ディレクトリを作成する
	os.makedirs(directory_path, exist_ok=True)

	args = {
    'dir': dir,
    'cycle' : 83,
	'Pos' : Pos,
	'zslices' : 51,
	'pixel_width' : 0.1300000,
	'voxel_depth' : 0.2000000}

	ij.py.run_macro(macro, args)

以下のマクロでは以下の操作を実行します。
* バッチモードの設定: 処理の高速化のためにバッチモードが有効にされています。
* イメージシーケンスの読み込み: 特定のディレクトリからイメージシーケンスを読み込み、ハイパースタックに変換します。
* 特定の領域の複製と測定: イメージの特定の領域を複製し、測定を行います。
* スライスの選択: 中間スライスを基準に、スライスの範囲を決定します。
* イメージのクロップ: 決定したスライス範囲に基づいてイメージをクロップします。
* ディスクリプタベースのシリーズ登録: イメージのシリーズに対してディスクリプタベースの登録を行い、画像の整合性を高めます。
* 再度の特定領域の複製と測定: さらにイメージの特定領域を複製し、測定を行います。
* 最小強度のZプロジェクション: 指定されたスライスで最小強度のZプロジェクションを行います。
* しきい値の設定と粒子解析: しきい値を設定し、マスクに変換後、粒子解析を行います。
* イメージの保存: 処理したイメージをTIFF形式で保存します。
* 終了処理: 開いたウィンドウを閉じて、ImageJを終了します。

In [ ]:
macro = """
#@ String dir
#@ int cycle
#@ int Pos
#@ int zslices
#@ float pixel_width
#@ float voxel_depth

setBatchMode("hide");

File.openSequence(dir+"/1_processed_images/1_FL_corrected/Pos"+IJ.pad(Pos, 2)+"/");
getDimensions(width, height, channels, slices, frames);
z_slices = slices/(4*cycle);
run("Stack to Hyperstack...", "order=xyczt(default) channels=4 slices="+z_slices+" frames="+cycle+" display=Composite");

rename("stack");

makeRectangle(25, 19, 111, 95);
run("Duplicate...", "duplicate channels=1");
run("Set Measurements...", "mean integrated redirect=None decimal=5");
run("Measure Stack...", "channels slices frames order=czt(default)");

//https://forum.image.sc/t/results-to-array/8727/3
RawIntDen_lower = newArray(nResults);
RawIntDen_upper = newArray(nResults);
//index_N = newArray(nResults);
Slice_N_low = newArray(nResults); 
Slice_N_upper = newArray(nResults); 

getDimensions(width, height, channels, slices, frames); 
med_slice_N = round(slices/ 2);
//print (med_slice_N);

for(i=nResults-1; i>-1; i--) {
	//index_N[i] = i;
	RawIntDen_lower[i] = getResult("RawIntDen", i);
	RawIntDen_upper[i] = getResult("RawIntDen", i);
	Slice_N_low[i] = getResult("Slice", i);
	Slice_N_upper[i] = getResult("Slice", i);
	if ( RawIntDen_lower[i] != 0 ){RawIntDen_lower = Array.deleteIndex(RawIntDen_lower, i);
	 								Slice_N_low = Array.deleteIndex(Slice_N_low, i);}
	if ( RawIntDen_upper[i] != 0 ){RawIntDen_upper = Array.deleteIndex(RawIntDen_upper, i);
	 								Slice_N_upper = Array.deleteIndex(Slice_N_upper, i);}
}

print(Slice_N_low.length);

for(i=Slice_N_low.length-1; i>-1; i--) {
	if ( Slice_N_low[i] > med_slice_N ){Slice_N_low = Array.deleteIndex(Slice_N_low, i);}
}

for(i=Slice_N_upper.length-1; i>-1; i--) {
	if ( Slice_N_upper[i] < med_slice_N ){Slice_N_upper = Array.deleteIndex(Slice_N_upper, i);}
}

Array.getStatistics(Slice_N_upper, min, max, mean, stdDev);
maxz = min
Array.getStatistics(Slice_N_low, min, max, mean, stdDev);
minz = max
print("Save from z"+minz+1+" to z"+maxz-1+".");

selectWindow("stack-1");
close();

selectWindow("stack");
run("Select None");
run("Duplicate...", "title=stack-1 duplicate slices="+minz+1+"-"+maxz-1+"");

selectWindow("stack");
close();

selectWindow("stack-1");
getDimensions(width, height, channels, slices, frames); 
med_slice_N = round(slices/ 2);

run("Duplicate...", "duplicate channels=1 slices="+med_slice_N+"");
run("Properties...", "channels=1 slices="+cycle+" frames=1 pixel_width="+pixel_width+" pixel_height="+pixel_width+" voxel_depth="+voxel_depth+"");
run("Z Project...", "projection=[Min Intensity]");


setThreshold(1, 65535);

// 画像の幅と高さを取得
width = getWidth();
height = getHeight();

// 中央の座標を計算
x = width / 2;
y = height / 2;

// ワンドツールを中央の座標に適用
doWand(x, y);


run("Convert to Mask");
run("Analyze Particles...", "size=0-Infinity");
selectWindow("stack-1");
run("Restore Selection");

selectWindow("MIN_stack-1-1");
close();
selectWindow("stack-1-1");
close();
selectWindow("stack-1");
run("Crop");

run("Clear Results");


rename("stack");

run("Descriptor-based series registration (2d/3d + t)", "series_of_images=stack brightness_of=Low approximate_size=[5 px] type_of_detections=[Minima & Maxima] subpixel_localization=[3-dimensional quadratic fit] transformation_model=[Affine (3d)] images_are_roughly_aligned number_of_neighbors=3 redundancy=1 significance=3 allowed_error_for_ransac=5 global_optimization=[All against first image (no global optimization)] range=5 choose_registration_channel=1 image=[Fuse and display] interpolation=[Linear Interpolation]");

selectWindow("stack");
close();




rename("stack");

makeRectangle(25, 19, 111, 95);
run("Duplicate...", "duplicate channels=1");
run("Set Measurements...", "mean integrated redirect=None decimal=5");
run("Measure Stack...", "channels slices frames order=czt(default)");

//https://forum.image.sc/t/results-to-array/8727/3
RawIntDen_lower = newArray(nResults);
RawIntDen_upper = newArray(nResults);
//index_N = newArray(nResults);
Slice_N_low = newArray(nResults); 
Slice_N_upper = newArray(nResults); 

getDimensions(width, height, channels, slices, frames); 
med_slice_N = round(slices/ 2);
//print (med_slice_N);

for(i=nResults-1; i>-1; i--) {
	//index_N[i] = i;
	RawIntDen_lower[i] = getResult("RawIntDen", i);
	RawIntDen_upper[i] = getResult("RawIntDen", i);
	Slice_N_low[i] = getResult("Slice", i);
	Slice_N_upper[i] = getResult("Slice", i);
	if ( RawIntDen_lower[i] != 0 ){RawIntDen_lower = Array.deleteIndex(RawIntDen_lower, i);
	 								Slice_N_low = Array.deleteIndex(Slice_N_low, i);}
	if ( RawIntDen_upper[i] != 0 ){RawIntDen_upper = Array.deleteIndex(RawIntDen_upper, i);
	 								Slice_N_upper = Array.deleteIndex(Slice_N_upper, i);}
}

print(Slice_N_low.length);

for(i=Slice_N_low.length-1; i>-1; i--) {
	if ( Slice_N_low[i] > med_slice_N ){Slice_N_low = Array.deleteIndex(Slice_N_low, i);}
}

for(i=Slice_N_upper.length-1; i>-1; i--) {
	if ( Slice_N_upper[i] < med_slice_N ){Slice_N_upper = Array.deleteIndex(Slice_N_upper, i);}
}

Array.getStatistics(Slice_N_upper, min, max, mean, stdDev);
maxz = min
Array.getStatistics(Slice_N_low, min, max, mean, stdDev);
minz = max
print("Save from z"+minz+1+" to z"+maxz-1+".");

selectWindow("stack-1");
close();

selectWindow("stack");
run("Select None");
run("Duplicate...", "title=stack-1 duplicate slices="+minz+1+"-"+maxz-1+"");

selectWindow("stack");
close();

selectWindow("stack-1");
getDimensions(width, height, channels, slices, frames); 
med_slice_N = round(slices/ 2);

run("Duplicate...", "duplicate channels=1 slices="+med_slice_N+"");
run("Properties...", "channels=1 slices="+cycle+" frames=1 pixel_width="+pixel_width+" pixel_height="+pixel_width+" voxel_depth="+voxel_depth+"");
run("Z Project...", "projection=[Min Intensity]");


setThreshold(1, 65535);

// 画像の幅と高さを取得
width = getWidth();
height = getHeight();

// 中央の座標を計算
x = width / 2;
y = height / 2;

// ワンドツールを中央の座標に適用
doWand(x, y);


run("Convert to Mask");
run("Analyze Particles...", "size=0-Infinity");
selectWindow("stack-1");
run("Restore Selection");

selectWindow("MIN_stack-1-1");
close();
selectWindow("stack-1-1");
close();
selectWindow("stack-1");
run("Crop");

run("Clear Results");




saveAs("Tiff", dir+"/1_processed_images/2_DBR_corrected/Pos_"+IJ.pad(Pos, 2)+".tif");
close("*");

run("Quit");
"""

# Assemble the directory path
directory_path = os.path.join(dir, "1_processed_images","2_DBR_corrected")

# Make the directory if it doesn't exist already
os.makedirs(directory_path, exist_ok=True)

# Repeats with Pos values from 1 to 3
for Pos in range(1, 4):  
	
	args = {
    'dir': dir,
    'cycle' : 83,
	'Pos' : Pos,
	'zslices' : 51,
	'pixel_width' : 0.1300000,
	'voxel_depth' : 0.2000000}

	ij.py.run_macro(macro, args)
 

以下のマクロでは以下の操作を実行します。
* イメージファイルの読み込み: 特定のディレクトリからイメージファイルを読み込み、"original"という名前で開きます。

* RNA-FISHの処理:
  *  最初のサイクルからlast_cycle_RNAseqFISHまでのフレームに対して、各チャネル（1から4）ごとに以下の処理を行います。
"original"ウィンドウから該当するチャネルとフレームの複製を作成し、"stack"と名付けます。
この複製をグレースケールに変換します。
変換されたイメージを特定のディレクトリにTIFF形式で保存します。ファイル名はフレームとチャネルに基づいています。
処理が終わったイメージを閉じます。

* DNA-FISHの処理:
  * last_cycle_RNAseqFISHの次のサイクルからlast_cycle_DNAseqFISHまでのフレームに対して、RNA-FISHと同様の処理を行いますが、保存するファイル名が異なります。
  
* seqIFの処理:
  * last_cycle_DNAseqFISHの次のサイクルからcycleまでのフレームに対して、RNA-FISHとDNA-FISHの処理と同様の手順を実行しますが、再び保存するファイル名が異なります。
全てのウィンドウを閉じる: 処理が完了したら、開いている全てのウィンドウを閉じます。

In [4]:
macro = """
#@ String dir
#@ int cycle
#@ int Pos
#@ int zslices
#@ float pixel_width
#@ float voxel_depth
#@ int last_cycle_RNAseqFISH
#@ int last_cycle_DNAseqFISH


    close("*");
	open(dir+"/1_processed_images/2_DBR_corrected/Pos_"+IJ.pad(Pos, 2)+".tif");
	
	rename("original");
	
	//RNA-FISH
	for (frames = 1; frames < (last_cycle_RNAseqFISH+1); frames++) {
		for (channels = 1; channels < 5; channels++) {
			selectWindow("original");
			
			run("Duplicate...", "title=stack duplicate channels="+channels+" frames="+frames+"");
			rename("stack");
			run("Grays");
	
			if (channels == 1) {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C4_TyR.tif");
			}else {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C"+(channels-1)+"_TyR.tif");
			}
			close();
		}
	}
	
	
	//DNA-FISH
	for (frames = last_cycle_RNAseqFISH+1; frames < (last_cycle_DNAseqFISH+1); frames++) {
		for (channels = 1; channels < 5; channels++) {
			selectWindow("original");
			
			run("Duplicate...", "title=stack duplicate channels="+channels+" frames="+frames+"");
			rename("stack");
			run("Grays");
	
			if (channels == 1) {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C4_TyD.tif");
			}else {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C"+(channels-1)+"_TyD.tif");
			}
			close();
		}
	}
	
	
	//seqIF
	for (frames = last_cycle_DNAseqFISH+1; frames < (cycle); frames++) {
		for (channels = 1; channels < 5; channels++) {
			selectWindow("original");
			
			run("Duplicate...", "title=stack duplicate channels="+channels+" frames="+frames+"");
			rename("stack");
			run("Grays");
	
	
			if (channels == 1) {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C4_TyI.tif");
			}else {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C"+(channels-1)+"_TyI.tif");
			}
			close();
		}
	}

close("*");


run("Quit");

"""


# Pos の値が 1 から 3 になるように繰り返す
for Pos in range(1, 4):  # range(1, 5) は 1, 2, 3, 4 の値を生成します
	# ImageJ マクロの 'IJ.pad(Pos, 2)' に相当する部分
	Pos_padded = str(Pos).zfill(2)

	# ディレクトリのパスを組み立てる
	directory_path = os.path.join(dir, "1_processed_images","3_diveded_files", "Pos" + Pos_padded)

	# ディレクトリを作成する
	os.makedirs(directory_path, exist_ok=True)
 
	args = {
	'dir': dir,
	'cycle' : 83,
	'Pos' : Pos,
	'zslices' : 51,
	'pixel_width' : 0.1300000,
	'voxel_depth' : 0.2000000,
	'last_cycle_RNAseqFISH' : 31,
	'last_cycle_DNAseqFISH' : 74}
 
	ij.py.run_macro(macro, args)
 

以下のマクロでは以下の操作を実行します。
* バッチモードの設定: ユーザーインターフェイスの更新を抑制し、処理速度を向上させるためにバッチモードが設定されています。

* 画像の読み込みと整形:
  * 特定の位置(Position)のDNA-seqFISH画像シーケンスを読み込みます。
  * 読み込んだ画像の次元を取得し、スライス数を計算します。
  * 画像スタックをハイパースタックに変換し、グレースケールで表示します。
* 平均強度プロジェクションの実行:
  * DNA-seqFISH画像のフレームをスライスに変換し、平均強度プロジェクションを行います。
  * これにより、fiducialマーカー部分が強調されます。
  * コントラストを強化します。
* 最大強度プロジェクションの実行:
  * 同じくDNA-seqFISH画像に最大強度プロジェクションを実行します。
  * これにより、すべての輝点の重ね合わせ像が得られます。
  * コントラストをさらに強化します。
  * 元のDNA-seqFISH画像は閉じます。
* 画像形式の変換:
  * 最大強度プロジェクションと平均強度プロジェクションの画像を32ビット形式に変換します。
* 画像の保存:
  * 平均強度プロジェクションの画像を"Fiducial_enhance_stack.tif"として保存します。
  * 最大強度プロジェクションの画像を"DNAseqFIS_foci_enhance_stack.tif"として保存します。
  * すべてのウィンドウを閉じます。

In [5]:
macro = """
#@ String dir
#@ int cycle
#@ int Pos
#@ int zslices
#@ float pixel_width
#@ float voxel_depth


setBatchMode("hide");
for (Position = 1; Position < Pos+1; Position++) {

	//setBatchMode("hide");
	

	//DNA-seqFISH画像を開く
	File.openSequence(dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Position, 2)+"/", " filter=_C3_TyD.tif start=2 count=40");
	rename("DNA_seqFISH");
	
	getDimensions(width, height, channels, slices, frames);
	z_slices = slices/(40);
 	
  run("Stack to Hyperstack...", "order=xyczt(default) channels=1 slices="+z_slices+" frames=40 display=Grayscale");


	//DNA-seqFISH画像のtをzに変換して、Average projectionする。すべての画像にfiducialマーカーがあるため、この操作によってfiducial marker部分が強調される
	selectWindow("DNA_seqFISH");
	run("Re-order Hyperstack ...", "channels=[Channels (c)] slices=[Frames (t)] frames=[Slices (z)]");
	run("Z Project...", "projection=[Average Intensity] all");
	run("Enhance Contrast", "saturated=0.01");


	//DNA-seqFISH画像のtをzに変換したものに対して、さらにMIP処理する。この操作は、すべての輝点の重ね合わせ像となる。
	selectWindow("DNA_seqFISH");
	run("Z Project...", "projection=[Max Intensity] all");
	run("Enhance Contrast", "saturated=0.35");
	selectWindow("DNA_seqFISH");
	close();


	run("Conversions...", " ");
	selectWindow("MAX_DNA_seqFISH");
	run("32-bit");
	selectWindow("AVG_DNA_seqFISH");
	run("32-bit");
	

	//setBatchMode("show");
	selectWindow("AVG_DNA_seqFISH");
	saveAs("Tiff", dir+"/1_processed_images/4_segmentation/Pos"+IJ.pad(Position, 2)+"/Fiducial_enhance_stack.tif");
	selectWindow("MAX_DNA_seqFISH");
	saveAs("Tiff", dir+"/1_processed_images/4_segmentation/Pos"+IJ.pad(Position, 2)+"/DNAseqFIS_foci_enhance_stack.tif");
	close("*");

}
"""

# Pos の値が 1 から 3 になるように繰り返す
for Pos in range(1, 4):  # range(1, 5) は 1, 2, 3, 4 の値を生成します
	# ImageJ マクロの 'IJ.pad(Pos, 2)' に相当する部分
	Pos_padded = str(Pos).zfill(2)

	# ディレクトリのパスを組み立てる
	directory_path = os.path.join(dir, "1_processed_images","4_segmentation", "Pos" + Pos_padded)

	# ディレクトリを作成する
	os.makedirs(directory_path, exist_ok=True)

	args = {
    'dir': dir,
    'cycle' : 83,
	'Pos' : Pos,
	'zslices' : 51,
	'pixel_width' : 0.1300000,
	'voxel_depth' : 0.2000000}

	ij.py.run_macro(macro, args)